In [1]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import string

In [2]:
data = pd.read_excel('data/review_full_text_vnmese_100.xlsx')

In [3]:
data.head()

,hotel_name,customer_name,title,full_content,rating,rating_New
0,Dal Vostro Hotel & Spa,Chrisinder,"Địa điểm, địa điểm, địa điểm ........",Vị trí tuyệt vời trong một con phố nhỏ cách Hồ...,bubble_50,5
1,Khách sạn Mường Thanh Grand Đà Nẵng,Lan thương,Đánh giá,Phục vụ tốt chăm sóc nhiệt tình. Buffer ngon n...,bubble_50,5
2,Khách sạn Sunny A,Kristel K,Giá tốt - chất lượng ở lại!,Chúng tôi tìm kiếm một kỳ nghỉ ngân sách thấp ...,bubble_50,5
3,Halong Palace Hotel,Chen M,Nghỉ đêm tại Vịnh Hạ Long,Phòng và phòng tắm rất sạch sẽ. Giường ngủ tho...,bubble_40,4
4,Khách sạn Riverside Sài Gòn,Nancy L,"Vị trí tuyệt vời, club lounge xứng đáng nhận đ...",Chúng tôi rất thích ở tại khách sạn Riverside ...,bubble_50,5


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   hotel_name     100 non-null    object
 1   customer_name  100 non-null    object
 2   title          100 non-null    object
 3   full_content   100 non-null    object
 4   rating         100 non-null    object
 5   rating_New     100 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


# Làm sạch dữ liệu
- Emojicon: ❤, 😍, 👍 …
- Teencode: ko -> không, ks -> khách sạn…
- Puntuations, Numbers : “”.!?#012

In [5]:
##LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
#print(teen_dict)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
#print(teen_dict)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
englist_lst = file.read().split('\n')
for line in englist_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
#print(teen_dict)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [6]:
for item in data['full_content'].head():
    print("#######")
    print(item)

#######
Vị trí tuyệt vời trong một con phố nhỏ cách Hồ Hoàn Kiếm vài phút đi bộ tuyệt vời vào buổi tối và thậm chí tốt hơn vào cuối tuần khi các con đường đóng cửa để giao thông thay đổi chào đón ở trung tâm của thành phố cổ. Băng qua đường bên phải và bạn tìm thấy một trong những quán bar tốt nhất ở Hà Nội, băng qua đường bên trái (sau khi bạn đặt chỗ) và bạn đang ở một trong những Nhà hàng Ý tốt nhất - một sự thay đổi đáng hoan nghênh cho những người như tôi đã mở rộng ở lại Dal Vostro ba lần vì. . . . . Dịch vụ khách hàng tốt nhất từ Daisy tại quầy lễ tân với nụ cười rạng rỡ của cô ấy với Jimmy, người sắp xếp các chuyến đi - không ngoại lệ trong Housekeep hoặc tại Breakfast, người ta luôn chào đón bằng một nụ cười - chân thật và luôn sẵn sàng giúp đỡ. Chỗ ở là nhỏ nhưng tốt và không…
#######
Phục vụ tốt chăm sóc nhiệt tình. Buffer ngon nhiều món. Phòng sạch sẽ.đầy đủ tiện nghi, view đẹp hướng ra cầu sông hàn.  Nhân viên thái độ phục vụ tốt. Vị trí trung tâm gần cầu rồng tiện cho di 

In [7]:
def process_text(text, dict_emoji, dict_teen, lst_wrong):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### CONVERT EMOJICON
        sentence = ''.join(dict_emoji[word]+' ' if word in dict_emoji else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(dict_teen[word] if word in dict_teen else word for word in sentence.split())
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        ###### DEL wrong words   
        sentence = ' '.join('' if word in lst_wrong else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '                    
    document = new_sentence  
    #print(document)
     ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [8]:
data['full_content_new'] = data['full_content'].apply(lambda x: process_text(str(x), emoji_dict, teen_dict, wrong_lst))

In [9]:
for item in data['full_content_new'].head():
    print("#######")
    print(item)

#######
vị trí tuyệt vời trọng một lừa phố nhỏ cách hồ hoàn kiếm vài phút đi bộ tuyệt vời vào buổi tối và thậm chí tốt hơn vào cuối tuần các lừa đường đóng cửa để thông đổi chào đón ở tâm của thành phố cổ. băng đường bên phải và bạn tìm thấy một trọng những quán quán tốt nhất ở hà nội băng đường bên trái bạn đặt chỗ và bạn đang ở một trọng những nhà hàng ý tốt nhất một sự đổi đáng nghênh cho những người như tôi đã mở rộng ở lại lần vì. . . . . dịch vụ khách hàng tốt nhất từ cúc tại quầy lễ tân với nụ cười rạng rỡ của cô ấy với người sắp xếp các chuyến đi không ngoại lệ trọng quản hoặc tại người luôn chào đón bằng một nụ cười chân thật và luôn sẵn sàng giúp đỡ. chỗ ở là nhỏ nhưng tốt và không.
#######
phục vụ tốt chăm sóc nhiệt tình. đệm nhiều món. phòng sạch sẽ đầy đủ tiện lượt đẹp hướng cầu sông hàn. nhân viên thái độ phục vụ tốt. vị trí tâm gần cầu rồng tiện cho di chuyển thăm quan.
#######
chúng tôi tìm kiếm một kỳ nghỉ ngân sách thấp chỉ trọng một đêm ở huế. chúng tôi thành lập nơi

# Chuẩn hóa Unicode Tiếng Việt

In [10]:
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [11]:
data['full_content_new'] = data['full_content_new'].apply(lambda x: covert_unicode(str(x)))

In [12]:
for item in data['full_content_new'].head():
    print("#######")
    print(item)

#######
vị trí tuyệt vời trọng một lừa phố nhỏ cách hồ hoàn kiếm vài phút đi bộ tuyệt vời vào buổi tối và thậm chí tốt hơn vào cuối tuần các lừa đường đóng cửa để thông đổi chào đón ở tâm của thành phố cổ. băng đường bên phải và bạn tìm thấy một trọng những quán quán tốt nhất ở hà nội băng đường bên trái bạn đặt chỗ và bạn đang ở một trọng những nhà hàng ý tốt nhất một sự đổi đáng nghênh cho những người như tôi đã mở rộng ở lại lần vì. . . . . dịch vụ khách hàng tốt nhất từ cúc tại quầy lễ tân với nụ cười rạng rỡ của cô ấy với người sắp xếp các chuyến đi không ngoại lệ trọng quản hoặc tại người luôn chào đón bằng một nụ cười chân thật và luôn sẵn sàng giúp đỡ. chỗ ở là nhỏ nhưng tốt và không.
#######
phục vụ tốt chăm sóc nhiệt tình. đệm nhiều món. phòng sạch sẽ đầy đủ tiện lượt đẹp hướng cầu sông hàn. nhân viên thái độ phục vụ tốt. vị trí tâm gần cầu rồng tiện cho di chuyển thăm quan.
#######
chúng tôi tìm kiếm một kỳ nghỉ ngân sách thấp chỉ trọng một đêm ở huế. chúng tôi thành lập nơi

# process_postag_thesea

In [13]:
def process_special_word(text):
    new_text = ''
    text_lst = text.split()
    i= 0
    if 'không' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
            #print(word)
            #print(i)
            if  word == 'không':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    else:
        new_text = text
    return new_text.strip()

In [14]:
def process_postag_thesea(text):
    new_document = ''
    for sentence in sent_tokenize(text):
        sentence = sentence.replace('.','')
        ###### POS tag
        lst_word_type = ['A','AB','V','VB','VY','R']
        sentence = ' '.join( word[0] if word[1].upper() in lst_word_type else '' for word in pos_tag(process_special_word(word_tokenize(sentence, format="text"))))
        new_document = new_document + sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [15]:
data['full_content_new'] = data['full_content_new'].apply(lambda x: process_postag_thesea(str(x)))

In [16]:
for item in data['full_content_new'].head():
    print("#######")
    print(item)

#######
tuyệt_vời trọng nhỏ cách hoàn_kiếm đi tuyệt_vời thậm_chí tốt hơn thông_đổi chào_đón cổ phải tìm thấy tốt nhất trái đặt đang ở tốt nhất đổi đáng nghênh đã mở_rộng ở lại tốt nhất lễ_tân cười rạng_rỡ sắp_xếp đi trọng_quản luôn chào_đón cười chân_thật luôn sẵn_sàng là nhỏ tốt không
#######
tốt đệm nhiều sạch_sẽ đẹp hàn phục_vụ tốt gần rồng_tiện di_chuyển thăm
#######
tìm_kiếm thấp chỉ_trọng thành_lập chúng_tôi rất hạnh_phúc rất tuyệt_gần đẹp rất sạch_sẽ đáng yêu cảm_thấy rất nghênh
#######
tắm rất sạch_sẽ ngủ gần lái đi đi dạo có ăn là lây
#######
rất thích sài gồm quyền_cập gồm ăn uống có không_cồn cũng có đã rất đáo gần là khởi_đầu đi quánh


# Remove Stopword

In [17]:
def remove_stopword(text, stopwords):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [18]:
data['full_content_new'] = data['full_content_new'].apply(lambda x: remove_stopword(str(x), stopwords_lst))

In [19]:
for item in data['full_content_new'].head():
    print("#######")
    print(item)

#######
tuyệt_vời hoàn_kiếm tuyệt_vời tốt thông_đổi chào_đón cổ tốt trái tốt đổi nghênh mở_rộng tốt cười rạng_rỡ sắp_xếp trọng_quản chào_đón cười chân_thật sẵn_sàng tốt không
#######
tốt đệm sạch_sẽ đẹp hàn phục_vụ tốt rồng_tiện di_chuyển thăm
#######
tìm_kiếm chỉ_trọng thành_lập hạnh_phúc tuyệt_gần đẹp sạch_sẽ yêu nghênh
#######
sạch_sẽ lái dạo lây
#######
thích sài quyền_cập uống không_cồn đáo khởi_đầu quánh


In [20]:
data.rating_New.describe()

count    100.000000
mean       4.430000
std        1.066051
min        1.000000
25%        4.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: rating_New, dtype: float64

In [21]:
data.rating_New.value_counts()

5    69
4    18
3     5
1     5
2     3
Name: rating_New, dtype: int64

In [42]:
# 1,2,3: không hài lòng
# 4, 5: hài lòng

In [25]:
data.head()

,hotel_name,customer_name,title,full_content,rating,rating_New,full_content_new
0,Dal Vostro Hotel & Spa,Chrisinder,"Địa điểm, địa điểm, địa điểm ........",Vị trí tuyệt vời trong một con phố nhỏ cách Hồ...,bubble_50,5,tuyệt_vời hoàn_kiếm tuyệt_vời tốt thông_đổi ch...
1,Khách sạn Mường Thanh Grand Đà Nẵng,Lan thương,Đánh giá,Phục vụ tốt chăm sóc nhiệt tình. Buffer ngon n...,bubble_50,5,tốt đệm sạch_sẽ đẹp hàn phục_vụ tốt rồng_tiện ...
2,Khách sạn Sunny A,Kristel K,Giá tốt - chất lượng ở lại!,Chúng tôi tìm kiếm một kỳ nghỉ ngân sách thấp ...,bubble_50,5,tìm_kiếm chỉ_trọng thành_lập hạnh_phúc tuyệt_g...
3,Halong Palace Hotel,Chen M,Nghỉ đêm tại Vịnh Hạ Long,Phòng và phòng tắm rất sạch sẽ. Giường ngủ tho...,bubble_40,4,sạch_sẽ lái dạo lây
4,Khách sạn Riverside Sài Gòn,Nancy L,"Vị trí tuyệt vời, club lounge xứng đáng nhận đ...",Chúng tôi rất thích ở tại khách sạn Riverside ...,bubble_50,5,thích sài quyền_cập uống không_cồn đáo khởi_đầ...


In [26]:
data['sentiment'] = np.where(data['rating_New']<=3, 1, 0)

In [31]:
data.head()

,hotel_name,customer_name,title,full_content,rating,rating_New,full_content_new,sentiment
0,Dal Vostro Hotel & Spa,Chrisinder,"Địa điểm, địa điểm, địa điểm ........",Vị trí tuyệt vời trong một con phố nhỏ cách Hồ...,bubble_50,5,tuyệt_vời hoàn_kiếm tuyệt_vời tốt thông_đổi ch...,0
1,Khách sạn Mường Thanh Grand Đà Nẵng,Lan thương,Đánh giá,Phục vụ tốt chăm sóc nhiệt tình. Buffer ngon n...,bubble_50,5,tốt đệm sạch_sẽ đẹp hàn phục_vụ tốt rồng_tiện ...,0
2,Khách sạn Sunny A,Kristel K,Giá tốt - chất lượng ở lại!,Chúng tôi tìm kiếm một kỳ nghỉ ngân sách thấp ...,bubble_50,5,tìm_kiếm chỉ_trọng thành_lập hạnh_phúc tuyệt_g...,0
3,Halong Palace Hotel,Chen M,Nghỉ đêm tại Vịnh Hạ Long,Phòng và phòng tắm rất sạch sẽ. Giường ngủ tho...,bubble_40,4,sạch_sẽ lái dạo lây,0
4,Khách sạn Riverside Sài Gòn,Nancy L,"Vị trí tuyệt vời, club lounge xứng đáng nhận đ...",Chúng tôi rất thích ở tại khách sạn Riverside ...,bubble_50,5,thích sài quyền_cập uống không_cồn đáo khởi_đầ...,0


In [43]:
X = data['full_content_new']
y = data['sentiment']

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

In [46]:
X_train_ = vect.fit_transform(X_train)
X_test_ = vect.transform(X_test)

In [47]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

In [48]:
nb.fit(X_train_, y_train)

MultinomialNB()

In [49]:
from sklearn.metrics import accuracy_score

print('Accuracy core of train:', accuracy_score(y_train, nb.predict(X_train_)))
print('Accuracy core of test:', accuracy_score(y_test, nb.predict(X_test_)))

Accuracy core of train: 1.0
Accuracy core of test: 0.85
